# Gem Puzzle

### Imports

In [1]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
import heapq

### Setup

In [2]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos_start', 'pos_end'])

def possible_moves(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_move(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos_start], new_state[action.pos_end] = new_state[action.pos_end], new_state[action.pos_start]
    return new_state

solved = np.arange(1, PUZZLE_DIM**2).tolist() + [0] # Desired result

def is_goal(state: np.ndarray) -> bool:
    return np.array_equal(state.flatten(), solved)

### Scrumble

In [3]:
RANDOMIZE_STEPS = 100
puzzle = np.array([i for i in range(1, PUZZLE_DIM ** 2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS)):
    puzzle = do_move(puzzle, choice(possible_moves(puzzle)))
puzzle

  0%|          | 0/100 [00:00<?, ?it/s]

array([[4, 8, 0],
       [6, 2, 5],
       [7, 1, 3]])

## Algorithms

### A*

In [4]:
def manhattan_distance(state: np.ndarray) -> int:
    dist = 0
    for i in range(PUZZLE_DIM):
        for j in range(PUZZLE_DIM):
            value = state[i, j]
            if value != 0:  # Ignore the blank tile
                target_x = (value - 1) // PUZZLE_DIM
                target_y = (value - 1) % PUZZLE_DIM
                dist += abs(target_x - i) + abs(target_y - j)
    return dist

def linear_conflict(state: np.ndarray) -> int:
    conflict = 0
    for row in range(PUZZLE_DIM):
        row_tiles = state[row, :]
        goal_positions = [(value - 1) % PUZZLE_DIM for value in row_tiles if value != 0]
        conflict += count_conflicts(goal_positions)

    for col in range(PUZZLE_DIM):
        col_tiles = state[:, col]
        goal_positions = [(value - 1) // PUZZLE_DIM for value in col_tiles if value != 0]
        conflict += count_conflicts(goal_positions)

    return conflict

def count_conflicts(positions: list[int]) -> int:
    conflicts = 0
    for i in range(len(positions)):
        for j in range(i + 1, len(positions)):
            if positions[i] > positions[j]:
                conflicts += 2
    return conflicts

def improved_heuristic(state: np.ndarray) -> int:
    return manhattan_distance(state) + linear_conflict(state)


In [5]:
import itertools

def a_star_solve(state: np.ndarray):
    open_set = []
    counter = itertools.count()  # Unique sequence count to prevent ambiguity
    heapq.heappush(open_set, (0, next(counter), state.copy(), [], 0))  # (priority, count, state, path, depth)
    visited = set()
    nodes_evaluated = 0

    while open_set:
        nodes_evaluated += 1
        _, _, current_state, steps, depth = heapq.heappop(open_set)

        if is_goal(current_state):
            return steps, nodes_evaluated

        state_tuple = tuple(current_state.flatten())
        if state_tuple in visited:
            continue
        visited.add(state_tuple)

        for act in possible_moves(current_state):
            new_state = do_move(current_state, act)
            new_state_tuple = tuple(new_state.flatten())
            if new_state_tuple in visited:
                continue

            # Calculate the cost: g (depth) + h (heuristic)
            g = depth + 1
            h = manhattan_distance(new_state)
            cost = g + h

            # Add the new state with a unique counter to the heap
            heapq.heappush(open_set, (cost, next(counter), new_state, steps + [new_state.copy()], g))

    return None

In [6]:
# Solve the puzzle using DFS
solution, nodes_evaluated = a_star_solve(puzzle)
if solution:
    print(f"Solution found in {len(solution)} steps:")
    print("Solution from:\n", puzzle, "\n to:\n", solution[-1])
    print("Nodes evaluated:", nodes_evaluated)
else:
    print("No solution found with the current starting state.")

Solution found in 20 steps:
Solution from:
 [[4 8 0]
 [6 2 5]
 [7 1 3]] 
 to:
 [[1 2 3]
 [4 5 6]
 [7 8 0]]
Nodes evaluated: 301


### IDDFS



In [7]:
def dfs_solve(state: np.ndarray, max_depth: int = 150):

    stack = [(state.copy(), [state.copy()], 0)]
    visited = set()

    while stack:
        current_state, steps, depth = stack.pop()

        if depth > max_depth:
            continue
        if is_goal(current_state):
            return steps

        # Mark the current state as visited
        state_tuple = tuple(current_state.flatten())
        if state_tuple in visited:
            continue
        visited.add(state_tuple)

        # Explore valid actions
        for act in possible_moves(current_state):
            new_state = do_move(current_state, act)

            if any(np.array_equal(new_state, visited_state) for visited_state in steps):
                continue

            stack.append((new_state, steps + [new_state.copy()], depth + 1))

    return None

In [8]:
def iddfs_solve(state: np.ndarray, max_depth: int = 150):
    for depth in range(max_depth + 1):
        result = dfs_solve(state, max_depth=depth)
        if result:
            return result
    return None

In [9]:
# Solve the puzzle using IDDFS
solution = iddfs_solve(puzzle)
if solution:
    print(f"Solution found in {len(solution)} steps:")
    for step in solution:
        print(step, "\n")
else:
    print("No solution found with the current starting state.")


Solution found in 35 steps:
[[4 8 0]
 [6 2 5]
 [7 1 3]] 

[[4 0 8]
 [6 2 5]
 [7 1 3]] 

[[0 4 8]
 [6 2 5]
 [7 1 3]] 

[[6 4 8]
 [0 2 5]
 [7 1 3]] 

[[6 4 8]
 [2 0 5]
 [7 1 3]] 

[[6 0 8]
 [2 4 5]
 [7 1 3]] 

[[6 8 0]
 [2 4 5]
 [7 1 3]] 

[[6 8 5]
 [2 4 0]
 [7 1 3]] 

[[6 8 5]
 [2 0 4]
 [7 1 3]] 

[[6 0 5]
 [2 8 4]
 [7 1 3]] 

[[0 6 5]
 [2 8 4]
 [7 1 3]] 

[[2 6 5]
 [0 8 4]
 [7 1 3]] 

[[2 6 5]
 [7 8 4]
 [0 1 3]] 

[[2 6 5]
 [7 8 4]
 [1 0 3]] 

[[2 6 5]
 [7 0 4]
 [1 8 3]] 

[[2 6 5]
 [7 4 0]
 [1 8 3]] 

[[2 6 5]
 [7 4 3]
 [1 8 0]] 

[[2 6 5]
 [7 4 3]
 [1 0 8]] 

[[2 6 5]
 [7 4 3]
 [0 1 8]] 

[[2 6 5]
 [0 4 3]
 [7 1 8]] 

[[2 6 5]
 [4 0 3]
 [7 1 8]] 

[[2 6 5]
 [4 1 3]
 [7 0 8]] 

[[2 6 5]
 [4 1 3]
 [0 7 8]] 

[[2 6 5]
 [0 1 3]
 [4 7 8]] 

[[2 6 5]
 [1 0 3]
 [4 7 8]] 

[[2 0 5]
 [1 6 3]
 [4 7 8]] 

[[2 5 0]
 [1 6 3]
 [4 7 8]] 

[[2 5 3]
 [1 6 0]
 [4 7 8]] 

[[2 5 3]
 [1 0 6]
 [4 7 8]] 

[[2 0 3]
 [1 5 6]
 [4 7 8]] 

[[0 2 3]
 [1 5 6]
 [4 7 8]] 

[[1 2 3]
 [0 5 6]
 [4 7 8]] 

[[1 2 3]
 [4